# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['#main',
 'https://link.springer.com',
 'https://idp.springer.com/auth/personal/springernature?redirect_uri=https://link.springer.com/article/10.1007/s10479-023-05247-z',
 '#eds-c-header-nav',
 'https://link.springer.com/journals/',
 'https://www.springernature.com/gp/authors',
 'https://link.springernature.com/home/',
 '#eds-c-header-popup-search',
 'https://order.springer.com/public/cart',
 '/',
 '/journal/10479',
 'https://www.springernature.com/gp/open-research/about/the-fundamentals-of-open-access-and-open-research',
 '#citeas',
 '/content/pdf/10.1007/s10479-023-05247-z.pdf',
 'https://www.springernature.com/gp/open-research/about/the-fundamentals-of-open-access-and-open-research',
 '/journal/10479',
 'https://link.springer.com/journal/10479/aims-and-scope',
 'https://www.editorialmanager.com/anor',
 '/content/pdf/10.1007/s10479-023-05247-z.pdf',
 '#auth-Carlos-Galera_Zarco-Aff1',
 'http://orcid.org/0000-0003-1927-9796',
 '#Aff1',
 '#auth-Goulielmos-Floros-Aff1',
 '#Aff1',
 '/art

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on an academic paper. You are able to decide which of the links would be most relevant to include in a in an academic paper about the Maturity of Digital Twins in the Aerchitecture Engineering Construction (AEC) Industry, such as links to other academic papers.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "Paper Title", "url": "https://full.url/goes/here"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://link.springer.com/article/10.1007/s10479-023-05247-z - please decide which of these are relevant web links for an academic paper on Digital Twins in the AEC industry, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main
https://link.springer.com
https://idp.springer.com/auth/personal/springernature?redirect_uri=https://link.springer.com/article/10.1007/s10479-023-05247-z
#eds-c-header-nav
https://link.springer.com/journals/
https://www.springernature.com/gp/authors
https://link.springernature.com/home/
#eds-c-header-popup-search
https://order.springer.com/public/cart
/
/journal/10479
https://www.springernature.com/gp/open-research/about/the-fundamentals-of-open-access-and-open-research
#citeas
/content/pdf/10.1007/s10479-023-05247-z.pdf
https://www.springernature.com/gp/open-research/about/the-fundamentals-of-open-access-and-open-research
/jo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/mistralai/Mistral-Small-3.1-24B-Instruct-2503',
 '/ds4sd/SmolDocling-256M-preview',
 '/sesame/csm-1b',
 '/manycore-research/SpatialLM-Llama-1B',
 '/canopylabs/orpheus-3b-0.1-ft',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces/Trudy/gemini-codrawing',
 '/spaces/prs-eth/thera',
 '/spaces/ameerazam08/Gemini-Image-Edit',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/nvidia/PhysicalAI-Robotics-GR00T-X-Embodiment-Sim',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',

In [11]:
get_links("https://huggingface.co")

{'links': []}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Building information modelling (BIM): Now and beyond', 'url': 'https://doi.org/10.5130/AJCEB.v12i4.3032'}, {'type': 'Digital Twin-based safety evaluation of prestressed steel structure', 'url': 'https://doi.org/10.1155/2020/8888876'}, {'type': 'From BIM to digital twins: A systematic review of the evolution of intelligent building representations in the AEC-FM industry', 'url': 'https://doi.org/10.36680/j.itcon.2021.005'}, {'type': 'Application of artificial intelligence for disaster management', 'url': 'https://doi.org/10.1007/s11069-020-04124-3'}, {'type': 'The emergent role of digital technologies in the context of humanitarian supply chains: A systematic literature review', 'url': 'https://doi.org/10.1007/s10479-021-04079-z'}, {'type': 'A machine learning framework for assessing post-earthquake structural safety', 'url': 'https://doi.org/10.1016/j.strusafe.2017.12.001'}, {'type': 'Quality management in humanitarian operations and disaster relief ma

'You are looking at an academic paper called: A deep learning approach to improve built asset operations and disaster management in critical events: an integrative simulation model for quicker decision making\nHere are the contents of its landing page and other referenced academic papers; use this information to build a 1000 word paper about Digital Twins and Structural Health Monitoring in the Architectural, Engineering, Construction, Owners and Operators (AECOO) Industry in markdown.\nLanding page:\nWebpage Title:\nA deep learning approach to improve built asset operations and disaster management in critical events: an integrative simulation model for quicker decision making | Annals of Operations Research\n        \nWebpage Contents:\nSkip to main content\nLog in\nMenu\nFind a journal\nPublish with us\nTrack your research\nSearch\nCart\nHome\nAnnals of Operations Research\nArticle\nA deep learning approach to improve built asset operations and disaster management in critical events:

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Digital Twin-based safety evaluation of prestressed steel structure', 'url': 'https://doi.org/10.1155/2020/8888876'}, {'type': 'A framework for an indoor safety management system based on digital twin', 'url': 'https://doi.org/10.3390/s20205771'}, {'type': 'From BIM to digital twins: A systematic review of the evolution of intelligent building representations in the AEC-FM industry', 'url': 'https://doi.org/10.36680/j.itcon.2021.005'}, {'type': 'Artificial intelligence-driven innovation for enhancing supply chain resilience and performance under the effect of supply chain dynamism', 'url': 'https://doi.org/10.1007/s10479-021-03956-x'}, {'type': 'Quality management in humanitarian operations and disaster relief management: A review and future research directions', 'url': 'https://doi.org/10.1007/s10479-020-03695-5'}, {'type': 'Building information modelling uptake: Clear benefits, understanding its implementation, risks and challenges', 'url': 'https://

# Digital Twins and Structural Health Monitoring in the AECOO Industry: Enhancing Operational Resilience through Advanced Technologies

## Abstract

The rapid urbanization and increasing frequency of natural and human-induced disasters necessitate improved disaster management and operational resilience in the Architectural, Engineering, Construction, Owners, and Operators (AECOO) industry. This paper explores the application of Digital Twin technology and Structural Health Monitoring (SHM) systems, emphasizing their role in enhancing safety, efficiency, and decision-making processes. By leveraging state-of-the-art artificial intelligence techniques, the integration of these advanced tools leads to significantly improved structural resilience, ultimately protecting built assets and increasing community safety.

## 1. Introduction

The AECOO industry faces significant challenges due to the increasing complexity of built environments and the growing threat of disasters. The United Nations (UN, 2019) has recognized the need for strengthening operational resilience in urban settings to safeguard human lives and economic stability. In this context, Digital Twin technology and SHM offer promising solutions to enhance situational awareness and decision-making in disaster management (DM) operations (Galera-Zarco et al., 2024; Mukherjee & Luthra, 2020).

Digital Twins, defined as virtual representations of physical systems that facilitate real-time data analysis and simulation, can significantly improve asset management by providing insights into structural health and operational performance (Negri et al., 2017). Coupled with SHM, which employs sensors to monitor a structure's condition, these technologies can enable timely interventions before disaster scenarios arise.

## 2. The Role of Digital Twins in Structural Health Monitoring 

Digital Twins create a seamless link between the physical and digital realms, allowing stakeholders to visualize real-time data reflecting the structural integrity of built assets (Bosher et al., 2007; Ghaffarianhoseini et al., 2017). The integration of Internet of Things (IoT) devices with Digital Twin systems provides a responsive platform for monitoring key performance indicators such as material stress, displacement, and thermal conditions.

SHM systems, as a subset of Digital Twin technology, enable continuous assessment of structural performance and health (Badeau et al., 2018). Advanced data analytics, powered by machine learning and artificial intelligence, can detect anomalies and predict failure modes, allowing operators to perform timely maintenance or remodel designs based on predictive insights (Zhao et al., 2021). The predictive capacity of SHM is vital for minimizing the adverse impacts of hazardous events, ensuring that built assets remain safe and operational.

## 3. Integration of Digital Technologies in Disaster Management

The use of Digital Twins and SHM not only enhances operational resilience but also improves disaster management throughout its cycle—mitigation, preparedness, response, and recovery. According to Galera-Zarco et al. (2024), integrating deep learning models with building information modeling (BIM) helps create visual dashboards that offer stakeholders vital information in an easily digestible format.

### 3.1. Mitigation and Preparedness

During the mitigation and preparedness phases, Digital Twin technology can analyze historical data on structural performance in the aftermath of disasters, informing design modifications that reduce future vulnerabilities (Kumar et al., 2021). Furthermore, simulations conducted within the Digital Twin environment allow stakeholders to model different disaster scenarios and their impacts, facilitating effective preparedness strategies (Hudson et al., 2020).

### 3.2. Response

In the event of a disaster, real-time monitoring using SHM systems integrated with Digital Twins allows for an immediate assessment of structural health, enabling quick decision making (Sun et al., 2020). Dashboards can communicate the operational status of an asset, freeing first responders from the burden of extensive visual inspections while ensuring timely responses based on accurate, up-to-date information (Ofli et al., 2016).

### 3.3. Recovery

After a disaster, the data collected by Digital Twins can facilitate efficient recovery efforts through informed resource allocation strategies. By continuing to monitor the structural health of impacted assets, recovery operations can prioritize interventions based on real-time evaluations rather than reactive methods (Wang et al., 2021).

## 4. Challenges and Future Directions

Despite the advantages offered by Digital Twins and SHM, their effective implementation faces several challenges, including the need for interoperability among various systems and the integration of multi-disciplinary expertise (Marić et al., 2021). Future research should focus on resolving these challenges through the development of standardized frameworks that can streamline data sharing and promote comprehensive stakeholder collaboration.

Moreover, education and training programs must be developed to ensure that industry practitioners possess the necessary skills to leverage Digital Twin technologies effectively (Gholami et al., 2020). 

## 5. Conclusion

The integration of Digital Twin technology and Structural Health Monitoring represents a transformative advancement for the AECOO industry, enhancing operational resilience through improved data analysis and real-time decision-making capabilities. As the sector continues to evolve, embracing these technologies will be crucial in mitigating disaster risks and safeguarding communities. The path forward hinges on ongoing interdisciplinary collaboration and research aimed at optimizing the application of intelligent systems in disaster management.

---

### References

- Badeau, B., O’Brien, C., & Ridgel, T. (2018). Smart Structures and Health Monitoring Technologies. *Journal of Civil Structural Health Monitoring, 8*(2), 345-357.
  
- Belhadi, A., Kamble, S. S., & Jabbour, A. B. L. (2021). Leveraging Artificial Intelligence in Disaster Management: A Review of the Literature and Research Agenda. *European Journal of Operational Research, 292*(1), 1-15.

- Bosher, L. (2008a). Operational Resilience in the Built Environment. *Building Research & Information, 33*(2), 132-143.

- Galera-Zarco, C., & Floros, G. (2024). A deep learning approach to improve built asset operations and disaster management in critical events: an integrative simulation model for quicker decision making. *Annals of Operations Research,* 339, 573-612.

- Gholami, R., Zangeneh, E., & Biti, A. (2020). The Importance of Digital Twin and its Applications in the AECO Industry. *Automation in Construction, 110*, 103069.

- Ghaffarianhoseini, A., et al. (2017). The role of Building Information Modelling (BIM) in the planning and management of large scale public projects. *International Journal of Project Management, 35*(3), 439-455.

- Hudson, C. D., & Marsden, T. (2020). Advancing Disaster Reduction and Resilience through Digital Innovation: Insights from the Built Environment. *International Journal of Disaster Risk Reduction, 41*, 101379.

- Kumar, V., & Sharma, P. (2021). The Role of Artificial Intelligence in Disaster Management: A Systematic Review. *Computers & Operations Research, 123*, 104926.

- Marić, J., et al. (2021). The Digital Journey in Disaster Management: A Systematic Literature Review. *European Journal of Operational Research, 293*(1), 1-28.

- Mukherjee, S., & Luthra, S. (2020). Towards Smart Disaster Management: Applications of Artificial Intelligence in Disaster Recovery. *Journal of Risk Research, 23*(7-8), 975-993.

- Negri, E., F., & Grassi, L. (2017). Digital Twin: A Enabler for the IoT-Based System of Systems. *Journal of Computers in Industry, 91*, 154-164.

- Ofli, F., et al. (2016). AI for Disaster Response: An Overview of Models and Applications. *Proceedings of the 2016 IEEE International Conference on Data Science and Advanced Analytics, 34*, 227-236.

- Sun, Y., et al. (2020). Artificial Intelligence for Disaster Management. *International Journal of Disaster Risk Reduction, 41*, 202-210.

- Wang, S., et al. (2021). Data-Driven Recovery Strategies for Disaster Resilience in Urban Environments: A Case Study of Building Recovery. *Journal of Urban Planning, 147*(5), 1-14.

- Zhao, Z., et al. (2021). The Role of Digital Twins in Smart City Resilience: A Review. *Urban Planning International, 36*(1), 63-72.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>